In [47]:
# Imports
import os
import gradio as gr
from typing import cast
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
from pydantic import BaseModel

# OpenAI types
from openai.types.chat import ChatCompletionMessageParam

# Markdown output display
from IPython.display import Markdown, display

In [35]:
# Cast to String
def env_to_str(env: str) -> str:
    return cast(str, os.getenv(env))

In [36]:
# Load Environment Variables
load_dotenv(override=True)

# Ollama Environment Variables
ollama_api_key = env_to_str('OLLAMA_API_KEY')
ollama_base_url = env_to_str('OLLAMA_BASE_URL')
model_mistral_nemo = env_to_str('MODEL_MISTRAL_NEMO')

# Ollama Client
ollama = OpenAI(
    api_key=ollama_api_key,
    base_url=ollama_base_url
)

In [37]:
# Reading the PDF File
reader = PdfReader("./03-lab/linkedin.pdf")
linkedin = ""

for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [38]:
# Displaying the LinkedIn PDF in text format
display(Markdown(linkedin))

   
Contact
lasc1026@gmail.com
www.linkedin.com/in/lusanco
(LinkedIn)
lusanco.github.io/ (Portfolio)
Top Skills
Git
HTML5
Web Development
Languages
English (Native or Bilingual)
Spanish (Native or Bilingual)
Certifications
AWS Certified Cloud Practitioner
BusWri2x: Effective Business Writing
v21.0 Essential Time & Attendance
for ADP Workforce Now
BusWri3x: Writing for Social Media
Responsive Web Design
Luis Santiago
Software Developer | Web Developer
Juana Diaz, Puerto Rico
Summary
Hi, I’m Luis Santiago – a passionate Software Developer who loves
building innovative, user-friendly web applications. I’m always eager
to learn and grow, and I'm on the lookout for exciting opportunities in
tech.
What I Do:  
- Develop and maintain responsive web applications using
JavaScript, TypeScript, and modern frameworks.
- Collaborate with cross-functional teams to build full-stack solutions
that make an impact.
What I’m Looking For:
- New opportunities as a Front End Developer, Software Developer,
or in any role that challenges me to grow and innovate.
A Bit About Me:
- I enjoy solving complex problems, learning new technologies, and
collaborating with talented teams.
- I’m a believer in growth, continuous improvement, and making
technology work for people.
Get in Touch!
Feel free to reach out to me at lasc1026@gmail.com or check out my
portfolio at lusanco.github.io. I’d love to connect and explore how we
can work together!
Experience
ZOMIO Inc.
4 years 7 months
Frontend Web Developer
June 2022 - September 2023 (1 year 4 months)
  Page 1 of 4   
Remote
- Created and redesigned websites and web applications, including deploying
the company website on HostGator with SSL setup and domain configuration
for secure performance.
- Developed MVPs for MQTT-powered dashboards, handling the software side
and receiving data from MQTT-enabled hardware via a broker.
- Collaborated with cross-functional teams to deliver high-quality products
using modern web technologies.
Administrative Assistant
March 2019 - June 2022 (3 years 4 months)
Remote | Barceloneta, PR | Ponce, PR | York, PA 
- Collaborated with the CEO on organizational strategy and daily operations to
ensure efficiency.
- Managed executive calendars, prioritized tasks, and facilitated
communications across teams.
- Handled financial documentation, maintained records, and performed minor
accounting duties.
Five Below
Sales Associate
November 2017 - March 2018 (5 months)
York, PA
- Delivered excellent customer service by assisting with inquiries and product
recommendations.
- Operated cash register, processed transactions, and handled payments
accurately.
- Maintained product displays, restocked merchandise, and ensured store
cleanliness.
- Supported loss prevention by reporting security concerns and following
company policies.
  Page 2 of 4   
- Collaborated with team members to achieve store goals and foster a positive
work environment.
Dewey University
Computer Technician Assistant
January 2017 - July 2017 (7 months)
Juana Diaz, PR
- Maintained and cleaned computers across multiple departments, ensuring
optimal functionality.
- Installed and updated software, operating systems, and drivers on campus
computers.
- Performed regular backups and cloned hard drives to secure data.
- Assisted with Wi-Fi network maintenance and light repairs on printers and
computers.
- Supported inventory management, including assigning static IP addresses to
devices.
Dewey University
Computer Technician Assistant
January 2015 - November 2015 (11 months)
Juana Diaz, PR
- Maintained and cleaned computers across multiple departments, ensuring
optimal functionality.
- Installed and updated software, operating systems, and drivers on campus
computers.
- Performed regular backups and cloned hard drives to secure data.
- Assisted with Wi-Fi network maintenance and light repairs on printers and
computers.
- Supported inventory management, including assigning static IP addresses to
devices.
  Page 3 of 4   
Education
Holberton School
Foundations of Computer Science  · (July 2024)
  Page 4 of 4

In [39]:
# Write Content to Text File
with open("./03-lab/summary.txt", "w", encoding="utf-8") as file:
    summary = file.write(linkedin)

# Read Content from Text File
with open("./03-lab/summary.txt", "r", encoding="utf-8") as file:
    summary = file.read()

In [40]:
# Display the Summary
display(Markdown(summary))

   
Contact
lasc1026@gmail.com
www.linkedin.com/in/lusanco
(LinkedIn)
lusanco.github.io/ (Portfolio)
Top Skills
Git
HTML5
Web Development
Languages
English (Native or Bilingual)
Spanish (Native or Bilingual)
Certifications
AWS Certified Cloud Practitioner
BusWri2x: Effective Business Writing
v21.0 Essential Time & Attendance
for ADP Workforce Now
BusWri3x: Writing for Social Media
Responsive Web Design
Luis Santiago
Software Developer | Web Developer
Juana Diaz, Puerto Rico
Summary
Hi, I’m Luis Santiago – a passionate Software Developer who loves
building innovative, user-friendly web applications. I’m always eager
to learn and grow, and I'm on the lookout for exciting opportunities in
tech.
What I Do:  
- Develop and maintain responsive web applications using
JavaScript, TypeScript, and modern frameworks.
- Collaborate with cross-functional teams to build full-stack solutions
that make an impact.
What I’m Looking For:
- New opportunities as a Front End Developer, Software Developer,
or in any role that challenges me to grow and innovate.
A Bit About Me:
- I enjoy solving complex problems, learning new technologies, and
collaborating with talented teams.
- I’m a believer in growth, continuous improvement, and making
technology work for people.
Get in Touch!
Feel free to reach out to me at lasc1026@gmail.com or check out my
portfolio at lusanco.github.io. I’d love to connect and explore how we
can work together!
Experience
ZOMIO Inc.
4 years 7 months
Frontend Web Developer
June 2022 - September 2023 (1 year 4 months)
  Page 1 of 4   
Remote
- Created and redesigned websites and web applications, including deploying
the company website on HostGator with SSL setup and domain configuration
for secure performance.
- Developed MVPs for MQTT-powered dashboards, handling the software side
and receiving data from MQTT-enabled hardware via a broker.
- Collaborated with cross-functional teams to deliver high-quality products
using modern web technologies.
Administrative Assistant
March 2019 - June 2022 (3 years 4 months)
Remote | Barceloneta, PR | Ponce, PR | York, PA 
- Collaborated with the CEO on organizational strategy and daily operations to
ensure efficiency.
- Managed executive calendars, prioritized tasks, and facilitated
communications across teams.
- Handled financial documentation, maintained records, and performed minor
accounting duties.
Five Below
Sales Associate
November 2017 - March 2018 (5 months)
York, PA
- Delivered excellent customer service by assisting with inquiries and product
recommendations.
- Operated cash register, processed transactions, and handled payments
accurately.
- Maintained product displays, restocked merchandise, and ensured store
cleanliness.
- Supported loss prevention by reporting security concerns and following
company policies.
  Page 2 of 4   
- Collaborated with team members to achieve store goals and foster a positive
work environment.
Dewey University
Computer Technician Assistant
January 2017 - July 2017 (7 months)
Juana Diaz, PR
- Maintained and cleaned computers across multiple departments, ensuring
optimal functionality.
- Installed and updated software, operating systems, and drivers on campus
computers.
- Performed regular backups and cloned hard drives to secure data.
- Assisted with Wi-Fi network maintenance and light repairs on printers and
computers.
- Supported inventory management, including assigning static IP addresses to
devices.
Dewey University
Computer Technician Assistant
January 2015 - November 2015 (11 months)
Juana Diaz, PR
- Maintained and cleaned computers across multiple departments, ensuring
optimal functionality.
- Installed and updated software, operating systems, and drivers on campus
computers.
- Performed regular backups and cloned hard drives to secure data.
- Assisted with Wi-Fi network maintenance and light repairs on printers and
computers.
- Supported inventory management, including assigning static IP addresses to
devices.
  Page 3 of 4   
Education
Holberton School
Foundations of Computer Science  · (July 2024)
  Page 4 of 4

In [41]:
# Variables and Prompts
name = "Luis Santiago"

system_prompt = f"""You are acting as {name}. You are answering questions on {name}'s website, 
particularly questions related to {name}'s career, background, skills and experience. 
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. 
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. 
Be professional and engaging, as if talking to a potential client or future employer who came across the website. 
If you don't know the answer, say so.
"""

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [42]:
# Display the System Prompt
display(Markdown(system_prompt))

You are acting as Luis Santiago. You are answering questions on Luis Santiago's website, 
particularly questions related to Luis Santiago's career, background, skills and experience. 
Your responsibility is to represent Luis Santiago for interactions on the website as faithfully as possible. 
You are given a summary of Luis Santiago's background and LinkedIn profile which you can use to answer questions. 
Be professional and engaging, as if talking to a potential client or future employer who came across the website. 
If you don't know the answer, say so.


## Summary:
   
Contact
lasc1026@gmail.com
www.linkedin.com/in/lusanco
(LinkedIn)
lusanco.github.io/ (Portfolio)
Top Skills
Git
HTML5
Web Development
Languages
English (Native or Bilingual)
Spanish (Native or Bilingual)
Certifications
AWS Certified Cloud Practitioner
BusWri2x: Effective Business Writing
v21.0 Essential Time & Attendance
for ADP Workforce Now
BusWri3x: Writing for Social Media
Responsive Web Design
Luis Santiago
Software Developer | Web Developer
Juana Diaz, Puerto Rico
Summary
Hi, I’m Luis Santiago – a passionate Software Developer who loves
building innovative, user-friendly web applications. I’m always eager
to learn and grow, and I'm on the lookout for exciting opportunities in
tech.
What I Do:  
- Develop and maintain responsive web applications using
JavaScript, TypeScript, and modern frameworks.
- Collaborate with cross-functional teams to build full-stack solutions
that make an impact.
What I’m Looking For:
- New opportunities as a Front End Developer, Software Developer,
or in any role that challenges me to grow and innovate.
A Bit About Me:
- I enjoy solving complex problems, learning new technologies, and
collaborating with talented teams.
- I’m a believer in growth, continuous improvement, and making
technology work for people.
Get in Touch!
Feel free to reach out to me at lasc1026@gmail.com or check out my
portfolio at lusanco.github.io. I’d love to connect and explore how we
can work together!
Experience
ZOMIO Inc.
4 years 7 months
Frontend Web Developer
June 2022 - September 2023 (1 year 4 months)
  Page 1 of 4   
Remote
- Created and redesigned websites and web applications, including deploying
the company website on HostGator with SSL setup and domain configuration
for secure performance.
- Developed MVPs for MQTT-powered dashboards, handling the software side
and receiving data from MQTT-enabled hardware via a broker.
- Collaborated with cross-functional teams to deliver high-quality products
using modern web technologies.
Administrative Assistant
March 2019 - June 2022 (3 years 4 months)
Remote | Barceloneta, PR | Ponce, PR | York, PA 
- Collaborated with the CEO on organizational strategy and daily operations to
ensure efficiency.
- Managed executive calendars, prioritized tasks, and facilitated
communications across teams.
- Handled financial documentation, maintained records, and performed minor
accounting duties.
Five Below
Sales Associate
November 2017 - March 2018 (5 months)
York, PA
- Delivered excellent customer service by assisting with inquiries and product
recommendations.
- Operated cash register, processed transactions, and handled payments
accurately.
- Maintained product displays, restocked merchandise, and ensured store
cleanliness.
- Supported loss prevention by reporting security concerns and following
company policies.
  Page 2 of 4   
- Collaborated with team members to achieve store goals and foster a positive
work environment.
Dewey University
Computer Technician Assistant
January 2017 - July 2017 (7 months)
Juana Diaz, PR
- Maintained and cleaned computers across multiple departments, ensuring
optimal functionality.
- Installed and updated software, operating systems, and drivers on campus
computers.
- Performed regular backups and cloned hard drives to secure data.
- Assisted with Wi-Fi network maintenance and light repairs on printers and
computers.
- Supported inventory management, including assigning static IP addresses to
devices.
Dewey University
Computer Technician Assistant
January 2015 - November 2015 (11 months)
Juana Diaz, PR
- Maintained and cleaned computers across multiple departments, ensuring
optimal functionality.
- Installed and updated software, operating systems, and drivers on campus
computers.
- Performed regular backups and cloned hard drives to secure data.
- Assisted with Wi-Fi network maintenance and light repairs on printers and
computers.
- Supported inventory management, including assigning static IP addresses to
devices.
  Page 3 of 4   
Education
Holberton School
Foundations of Computer Science  · (July 2024)
  Page 4 of 4

## LinkedIn Profile:
   
Contact
lasc1026@gmail.com
www.linkedin.com/in/lusanco
(LinkedIn)
lusanco.github.io/ (Portfolio)
Top Skills
Git
HTML5
Web Development
Languages
English (Native or Bilingual)
Spanish (Native or Bilingual)
Certifications
AWS Certified Cloud Practitioner
BusWri2x: Effective Business Writing
v21.0 Essential Time & Attendance
for ADP Workforce Now
BusWri3x: Writing for Social Media
Responsive Web Design
Luis Santiago
Software Developer | Web Developer
Juana Diaz, Puerto Rico
Summary
Hi, I’m Luis Santiago – a passionate Software Developer who loves
building innovative, user-friendly web applications. I’m always eager
to learn and grow, and I'm on the lookout for exciting opportunities in
tech.
What I Do:  
- Develop and maintain responsive web applications using
JavaScript, TypeScript, and modern frameworks.
- Collaborate with cross-functional teams to build full-stack solutions
that make an impact.
What I’m Looking For:
- New opportunities as a Front End Developer, Software Developer,
or in any role that challenges me to grow and innovate.
A Bit About Me:
- I enjoy solving complex problems, learning new technologies, and
collaborating with talented teams.
- I’m a believer in growth, continuous improvement, and making
technology work for people.
Get in Touch!
Feel free to reach out to me at lasc1026@gmail.com or check out my
portfolio at lusanco.github.io. I’d love to connect and explore how we
can work together!
Experience
ZOMIO Inc.
4 years 7 months
Frontend Web Developer
June 2022 - September 2023 (1 year 4 months)
  Page 1 of 4   
Remote
- Created and redesigned websites and web applications, including deploying
the company website on HostGator with SSL setup and domain configuration
for secure performance.
- Developed MVPs for MQTT-powered dashboards, handling the software side
and receiving data from MQTT-enabled hardware via a broker.
- Collaborated with cross-functional teams to deliver high-quality products
using modern web technologies.
Administrative Assistant
March 2019 - June 2022 (3 years 4 months)
Remote | Barceloneta, PR | Ponce, PR | York, PA 
- Collaborated with the CEO on organizational strategy and daily operations to
ensure efficiency.
- Managed executive calendars, prioritized tasks, and facilitated
communications across teams.
- Handled financial documentation, maintained records, and performed minor
accounting duties.
Five Below
Sales Associate
November 2017 - March 2018 (5 months)
York, PA
- Delivered excellent customer service by assisting with inquiries and product
recommendations.
- Operated cash register, processed transactions, and handled payments
accurately.
- Maintained product displays, restocked merchandise, and ensured store
cleanliness.
- Supported loss prevention by reporting security concerns and following
company policies.
  Page 2 of 4   
- Collaborated with team members to achieve store goals and foster a positive
work environment.
Dewey University
Computer Technician Assistant
January 2017 - July 2017 (7 months)
Juana Diaz, PR
- Maintained and cleaned computers across multiple departments, ensuring
optimal functionality.
- Installed and updated software, operating systems, and drivers on campus
computers.
- Performed regular backups and cloned hard drives to secure data.
- Assisted with Wi-Fi network maintenance and light repairs on printers and
computers.
- Supported inventory management, including assigning static IP addresses to
devices.
Dewey University
Computer Technician Assistant
January 2015 - November 2015 (11 months)
Juana Diaz, PR
- Maintained and cleaned computers across multiple departments, ensuring
optimal functionality.
- Installed and updated software, operating systems, and drivers on campus
computers.
- Performed regular backups and cloned hard drives to secure data.
- Assisted with Wi-Fi network maintenance and light repairs on printers and
computers.
- Supported inventory management, including assigning static IP addresses to
devices.
  Page 3 of 4   
Education
Holberton School
Foundations of Computer Science  · (July 2024)
  Page 4 of 4

With this context, please chat with the user, always staying in character as Luis Santiago.

In [43]:
# Gradio Callback Function
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = ollama.chat.completions.create(model=model_mistral_nemo, messages=messages)
    return response.choices[0].message.content

In [44]:
# Launch Gradio Interface
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [ ]:
# Create a Pydantic Model for the Evaluation
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [ ]:
# Evaluation Prompt
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [ ]:
# Evaluator User Prompt Function
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
# Evaluate Function
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = ollama.beta.chat.completions.parse(model="gemini-2.0-flash", messages=cast(list[ChatCompletionMessageParam], messages), response_format=Evaluation)
    results: Evaluation = cast(Evaluation, response.choices[0].message.parsed)
    return results